# Домашнее задание № 1

In [47]:
!pip install pymystem3
!pip install pymorphy2[fast]
!pip install razdel
!pip install gensim
!pip install nltk
!pip install rusenttokenize
!pip install regex

  Using cached https://files.pythonhosted.org/packages/b8/ef/91b619a399685f7a0a95a03628006ba814d96293bbbbed234ee66fbdefd9/DAWG-0.8.0.tar.gz
  Running setup.py clean for DAWG
Failed to build DAWG
  Running setup.py install for DAWG: started
    Running setup.py install for DAWG: finished with status 'error'


  ERROR: Complete output from command 'C:\Users\sara7\Anaconda3\python.exe' -u -c 'import setuptools, tokenize;__file__='"'"'C:\\Users\\sara7\\AppData\\Local\\Temp\\pip-install-8xkg_my9\\DAWG\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\sara7\AppData\Local\Temp\pip-wheel-uw3o0ugw' --python-tag cp37:
  ERROR: running bdist_wheel
  running build
  running build_ext
  building 'dawg' extension
  error: Microsoft Visual C++ 14.0 is required. Get it with "Microsoft Visual C++ Build Tools": https://visualstudio.microsoft.com/downloads/
  ----------------------------------------
  ERROR: Failed building wheel for DAWG
    ERROR: Complete output from command 'C:\Users\sara7\Anaconda3\python.exe' -u -c 'import setuptools, tokenize;__file__='"'"'C:\\Users\\sara7\\AppData\\Local\\Temp\\pip-install-8xkg_my9\\DAWG\\setup.py'"'"';f=getattr(tokeni

In [48]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymystem3 import Mystem
from pymorphy2 import MorphAnalyzer
import pandas as pd
import json
from razdel import sentenize

## 1. Задание на удаление тегов (2 балла)

Часто в данных, с которыми нам нужно работать помимо текста присутствует ещё какая-то лишняя информация: тэги, ссылки, код, разметка. Она, конечно, не всегда лишняя, но обычно от неё лучше избавиться.

Возьмем в качестве примера статью с Хабрахабра. Она скачана автоматически и в там остались некоторые тэги.

In [49]:
text = '''
Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — <a href="http://www.crazyegg.com" title="Сумасшедшие яйца">CrazyEgg</a>. Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо. Запоминается? Отлично!<br><br><img src="http://img172.imageshack.us/img172/8434/18274658kc4.png" alt="Сумасшедшее яйцо"><br><br><blockquote><h3>Что это такое?</h3><br>
Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.<br>
Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).<br><br><img src="https://habrastorage.org/getpro/habr/post_images/7c7/298/33c/7c729833cd942cc493e68833e3e0f12d.jpg" alt="Тепловое отслеживание популярности"><br></blockquote><br><br><a name="habracut"></a><br><br><blockquote><h3>Для кого это?</h3><br>
Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуитивный интерфейс, и хороший дизайн дает о себе знать — сервис массово популяризировался пол года-год назад. Относительно недорогой, но уж точно не из дешевых — сервис предполагает собой 4 платных линии и одну бесплатную. Дабы написать этот обзор я не поленился заплатить 19 долларов (в месяц) выбрав средний вариант — для нескольких проектов с включенными дополнительными функциями.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/0b0/0a0/b16/0b00a0b16b1eda28e35f39487dcd2545.jpg" alt="Отслеживание ссылок"></blockquote><br><blockquote><img src="https://habrastorage.org/getpro/habr/post_images/0b5/433/892/0b54338921665ffb5a90930147296f5b.jpg" alt="Список по популярности"><br><br>
Да-да, сервис не бесплатен. Точнее бесплатная возможность потестировать есть, но она немного обрезана (можно отслеживать только 5000 посетителей и всего 4 страницы на сайте (внимание — 4 страницы, а не сайтов), т.е. вполне хватает для того, чтобы понять полезность сервиса). Зачем это?</h3><br>
С помощью этого сервиса можно тасовать блоки на сайте, которые полезны пользователям больше всего. Больше не нужно спорить создателям — какой блок где расположить. В этом им поможет <a href="http://ttp://www.crazyegg.com" title="Сумасшедшие яйца">Сумасшедшее яйцо</a>.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/38c/af8/e75/38caf8e753782a01dc6419d3902edd57.jpg" alt="Добавление проекта"></blockquote><br><blockquote>Также этот сервис поможет вам понять — в какой зоне сайта лучше всего располагать рекламу, когда в вашем сервисе речь зайдет о монетизации. Ведь альтруизм это хорошо, а деньги на содержание сервиса нужны, и не лишним будет вычислить зоны где реклама будет приносить наибольшую отдачу, и наименьшее раздражение у пользователей.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/0dd/3fb/b34/0dd3fbb34709e77f3fbcd6523c7eac77.jpg" alt="еще полезности"></blockquote><br><blockquote><h3>Как это работает?</h3><br>
Никаких километровых скриптов вставлять не нужно, достаточно вставиь 2 строчки яваскрипта, и сервис начнет отслеживание. Насколько я понял — исполнительный скрипт работает на сервере <a href="http://www.crazyegg.com" title="Сумасшедшие яйца">CrazyEgg</a>, поэтому ваш сайт от этого в производительности не потеряет ни секунды, а полезность довольно таки большая.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/a90/e4e/e67/a90e4ee6760978463e6306a2f5982e24.jpg" alt="управление популярностью"></blockquote><br><blockquote><h3>Сколько это стоит?</h3><br>
Как я уже упоминал — сервис далеко не бесплатен, хоть и имеет тестовую-бесплатную версию. Расскажу подробнее о тарификации.</blockquote><br><blockquote>1. Бесплатная тестовая версия. Включает в себя возможность отслеживания 5 000 посещений, 4 страницы на сайте.<br><br>
2. Базовая версия. В этой версии можно отследить 10 000 посещений, и 10 страниц, что вполне достаточно для среднего корпоративного сайта. Стоит базовый комплект — 9 долларов в месяц. В этот комплект включены все дополнительные функции.<br><br>
3. Версия «Стандарт». В неё входит возможность отслеживания 25 000 посещений на 20 страницах. Вполне подходит для тестирования нового стартапа. Стоит она 19 долларов в месяц, именно её я купил для тестирования сервиса, и написания этого обзора.<br><br>
4. Версия «Плюс». Отличается от предыдущей возможностью отслеживания 100 000 посещений, 50 страниц. Очень хороший тариф для крупных сервисов. Стоит 49 долларов в месяц. Довольно большие деньги за сервис, но они обычно с лихвой окупаются.<br><br>
5. Версия «Про». Стоит почти 100 долларов, имеет возможность отследить 250 000 посещений на ста страницах. Тариф подходит для монстров с большой посещаемостью и большим количеством страниц.<br><br><img src="https://habrastorage.org/getpro/habr/post_images/f73/fb2/62d/f73fb262da4618a8dde67690cfd191ea.jpg" alt="Отслеживание статистики"></blockquote><br><blockquote><h3>Есть и аналоги</h3><br>
Я не поленился, и собрал еще пару ссылок с аналогами, которые предлагают такие же услуги, но немного дешевле.</blockquote><br><blockquote> 1. <a href="https://www.google.com/analytics/home/?hl=en" title="Шикарный сбор и анализ статистики">Google Analytics</a> — бесплатный сервис для сбора и анализа статистики, вывод статистики в наиболее наглядной форме, и без разнообразных рейтингов, счетчиков. Очень подробная и полезная вещь. Рекомендую, т.к. сам пользуюсь им для этого блога.<br><br>
2. <a href="http://www.mapsurface.com/" title="Сервис для отслеживания популярности блоков на сайте">MapSurface</a> — сам еще не использовал (т.к. предпочел CrazyEgg), но врядли будучи плохим сервис собрал бы множество положительных отзывов. К сожалению он сейчас находится в статусе закрытой беты.</blockquote><br><blockquote><h3>Вывод</h3><br>
Использовать можно, и нужно. Вот только тарифы довольно больно кусаются, но обычно эти деньги потом с лихвой отбиваются на повышении конвертации посетителей в деньги. Использовать сервис нужно для тестирования рекламных мест и удобства отдельных страниц, что помогает опять же повысить конвертацию. В общем и целом — <a href="http://ttp://www.crazyegg.com">полезная вещь</a> для каждого владельца сайтов, а для юзабилиста вообще практически обязательна. К счастью для людей, которые поиздержались деньгами в этом месяце — есть полезные аналоги.<br><br><img src="http://img241.imageshack.us/img241/2346/confettitnml5.jpg" alt="Интересная идея визуализации популярности - конфети"></blockquote><br><br>
Автор: <a href="http://www.birzool.com/" title="Я пишу о юзабилити веб интерфейсов">Ярослав Бирзул</a> (DezmASter).<br>
Источник: <a href="http://www.birzool.com/crazyegg/" title="Сумасшедшие яйца, первоисточник">Блог о юзабилити веб интерфейсов</a>.<br><br>
PS: Всех с прошедшим Новым годом! От всей души желаю вам всего самого-самого лучшего, чего вы желаете только в самых сокровенных мечтах. Удачно вам провести время.
'''

С помощью регулярных выражение очистите текст от тэгов (мета информацию внутри тэгов тоже удалите). В получившемся тексте точки не должны слипаться с началом следующего предложений, также не должно быть лишних пробелов.

In [50]:
def remove_tags(t):
    sepr_ttl_t = re.sub('<h3>', '\n', t) # отделяем заголовки от абзацев, чтобы не "слипались"
    cln_t = re.sub('<.+?>',r'', sepr_ttl_t)
    return cln_t

In [51]:
clean_text = remove_tags(text)
print(clean_text)


Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg. Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо. Запоминается? Отлично!
Что это такое?
Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.
Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).
Для кого это?
Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуитивный интерфейс, и хороший дизайн 

## 2. Задание на выбор слов после "не " (1 балл)

Вытащите из очищенного текста все слова, которые идут после частицы "не" (случаи, когда "не" - часть предыдущего слова, не должны извлекаться), **кроме слов, начинающихся на гласную букву**. Сама часть "не " не должна извлекаться.

In [52]:
re.findall('\W[Нн]е ([^аеёиоуыэюя][а-я]+)', clean_text)

['знаю',
 'поленился',
 'бесплатен',
 'сайтов',
 'нужно',
 'лишним',
 'нужно',
 'потеряет',
 'бесплатен',
 'поленился']

## 3. Задание на выбор глаголов (1 балл)

Попробуйте вытащить из очищенного текста как можно больше глаголов (минимум 40 с повоторениями), не перечисляя их в паттерне полностью. Можете токенизировать текст и использовать re.match, а можете использовать re.findall с \\b в начале и конце паттерна, чтобы не матчить части слов.

В результате не должно быть не глаголов. Также постарайтесь сделать паттерн максимально коротким.

In [53]:
verbs = re.findall('\\b[А-Яа-я]+(?:аю|[еиюая](?:т[ье]?|л))\\b', clean_text)
print(len(verbs), verbs)

62 ['будет', 'знаю', 'работает', 'позволяет', 'отслеживать', 'выводить', 'совмещает', 'может', 'дает', 'знать', 'предполагает', 'написать', 'заплатить', 'потестировать', 'отслеживать', 'хватает', 'понять', 'тасовать', 'спорить', 'расположить', 'поможет', 'поможет', 'понять', 'располагать', 'зайдет', 'будет', 'вычислить', 'будет', 'приносить', 'работает', 'вставлять', 'начнет', 'понял', 'работает', 'потеряет', 'стоит', 'упоминал', 'имеет', 'Включает', 'отследить', 'Стоит', 'входит', 'подходит', 'Стоит', 'купил', 'Стоит', 'Стоит', 'имеет', 'отследить', 'подходит', 'собрал', 'предлагают', 'использовал', 'предпочел', 'собрал', 'Использовать', 'Использовать', 'помогает', 'опять', 'повысить', 'желаю', 'желаете']


_(Пришлось отказаться от некоторых своих идей в пользу корректного выполнения задания, то есть я не учитываю определенные варианты окончаний в коде (такие как -им/ем, -ишь/ешь, -у, -ую, -ул и проч.) не потому что не думаю о них, а чтобы соблюсти условия задания. Чтобы сократить паттерн, я на разных этапах работы я смотрела на получающийся список и корректировала его, если видела, что таких форм глаголов в тексте всё равно нет; также я корректировала его, если понимала, что таких форм глаголов довольно мало, но при этом вместе с ними неизбежно захватываются ненужные слова... эх)_

## 4. Pymorphy vs Mystem (2 балла)

Лемматизируйте очищенный текст с помощью mystem и pymorphy. Сравните результаты: есть ли разница, что лучше. 

Для пайморфи используйте токенизацию из mystem, чтобы исключить влияние токенизации на результат.

In [54]:
# токенизация (общая для обоих методов)

tokens = word_tokenize(clean_text)

# очистка от пунктуации
punct = '!"#$%&()*\+,-\./:;<=>?@\[\]^_`{|}~„“«»†*\—/\-‘’'
words = [w.strip(punct) for w in tokens]

# выравнивание регистра и удаление пустых элементов
small_words = [w.lower() for w in words if w != '']

# очистка от стоп-слов

sw = stopwords.words('russian')

no_stop_words = [w for w in small_words if w not in sw]

# очистка от чисел 
list_of_words = []
for n in no_stop_words:
    new_w = re.sub('[0-9]',r'', n)
    if new_w !='':
        list_of_words.append(new_w)
    
# преобразование в строку для Mystem   
text_of_words = ' '.join(list_of_words)

#### Mystem

In [55]:
m = Mystem()
lemmas = m.lemmatize(text_of_words)

mystem_result = []
for lem in lemmas:
    if lem !=' ':
        mystem_result.append(lem)

In [56]:
len(mystem_result)

470

#### Pymorphy

In [57]:
morph = MorphAnalyzer()

pymorphy_result =[]
for w in list_of_words:
    p = morph.parse(w)
    pymorphy_result.append(p[0].normal_form)

In [58]:
len(pymorphy_result)

452

#### Сравниваем:

In [59]:
tabledata = []
for i in range(len(pymorphy_result)): #т.к. у pymorphy результатов меньше
    tabledata.append([i, mystem_result[i], pymorphy_result[i]])

In [60]:
pd.DataFrame(tabledata, columns=['№','mystem_result', 'pymorphy_result'])

,№,mystem_result,pymorphy_result
0,0,сегодняшний,сегодняшний
1,1,заметка,заметка
2,2,сервис,сервис
3,3,отслеживание,отслеживание
4,4,активность,активность
5,5,пользователь,пользователь
6,6,crazyegg,crazyegg
7,7,знать,знать
8,8,кто,кома
9,9,обязанный,обязать


_Где-то на отрезке 90:120 можно видеть,как списки перестали совпадать из-за того, что mystem, видимо, ещё дополнительно делит внутри себя слова, которые пишутся через дефис, чего не делала заданная выше общая токенизация... и чего не делает pymorphy! времени доработать это немного не хватает, да и 90 первых параллельных сравнений достаточно для сравнения: Mystem, справляется лучше, реже путает части речи у омонимичных форм (за счёт опоры на контекст, видимо..)_

## 5. Разбиение на предложения в Mystem (2 балла)

С помощью Mystem разделите текст из первого задания (очищенный) на предложения. Сравните разбиение с razdel.sentenize

#### Mystem

In [61]:
f = open('text.txt', 'w', encoding="utf-8")
f.write(clean_text)
f.close()

In [62]:
!mystem -isc --format json text.txt text_parsed.txt

In [63]:
t = [json.loads(line) for line in open('text_parsed.txt', encoding="utf8")]

In [64]:
for i in range(len(t)):
    sent = [parse['text'] for parse in t[i] if parse.get('analysis')]
    print(' '.join(sent))


Сегодняшняя заметка будет о сервисе отслеживания активности пользователя Я не знаю кому обязан сервис таким говорящим именем но оно работает и хорошо Запоминается Отлично
Что это такое
Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту кто куда кликнул какие ссылки наиболее популярные и тому подобная разнородная информация для юзабилистов
Сервис позволяет отслеживать активность определенных пользователей и выводить эти данные в различных формах инфракрасная где чем активнее область тем она теплее салюты чем активнее область тем больше конфети простой список с сортировкой по активности колбы чем заполненнее колба тем активнее область облака на мой взгляд наиболее удобный вариант совмещает в себе все остальные вместе взятые
Для кого это
Разумеется сервис изначально планировался как первый помощник для юзабилистов но пользоваться им может каждый у кого есть деньги интуитивный интерфейс и хороший дизайн дает о себе знать сервис массово популяризировался по

#### razdel.sentenize

In [65]:
sents = list(sentenize(clean_text))
for sent in sents:
    print(sent.text)

Сегодняшняя заметка будет о сервисе отслеживания активности пользователя — CrazyEgg.
Я не знаю кому обязан сервис таким говорящим именем, но оно работает, и хорошо.
Запоминается?
Отлично!
Что это такое?
Как уже сказано выше это сервис для отслеживания перемещения пользователей по сайту — кто куда кликнул, какие ссылки наиболее популярные и тому подобная, разнородная информация для юзабилистов.
Сервис позволяет отслеживать активность определенных пользователей, и выводить эти данные в различных формах: «инфракрасная» — где чем активнее область, тем она «теплее», салюты (чем активнее область, тем больше конфети), простой список с сортировкой по активности, колбы (чем заполненнее колба, тем активнее область), облака (на мой взгляд наиболее удобный вариант — совмещает в себе все остальные вместе взятые).
Для кого это?
Разумеется сервис изначально планировался как первый помощник для юзабилистов, но пользоваться им может каждый, у кого есть деньги — интуитивный интерфейс, и хороший дизайн д

_В mystem-е некоторые предложения слиплись, пропали все знаки препинания (наверное, для данной задачи это не хорошо), razdel.sentenize справился лучше._

## 6*. Дизамбигуация в Mystem (2 балла)

*необязательное задание на 10 баллов

Одна из фичей майстема - контекстная дизамбигуация (т.е. определение правильного разбора в зависимости от контекста).

Запустите mystem на очищенном тексте с дизамбигуацией и без и проверьте есть ли какие-то различия.

_Я не нашла, как отключать дизамбигуацию для mystem-а какими-то прямыми методами, но её явно не будет, если давать mystem-у на вход не предложения или весь текст, а слова по отдельности; например, как для pymorphy: список поэлементно_

In [66]:
mystem_no_diz_result = []
for w in list_of_words:
    no_diz = m.analyze(w)
    mystem_no_diz_result.append(no_diz)

print(mystem_no_diz_result)

KeyboardInterrupt: 

_Чтожж, оказалось, что это невозможно_ ¯\_(ツ)_/¯

_Тогда я не поняль, как это сделать, объясните пожалуйста_